In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## added selenium code
from selenium import webdriver
import time
from bs4 import BeautifulSoup
## For parallel execution
from multiprocessing import Pool
from tqdm.auto import tqdm
from datetime import date

today = date.today().strftime('%Y-%m-%d')

In [2]:
#Scrape data from all pages
# Enable javascript and disable automation flag
options = webdriver.ChromeOptions()
options.add_argument("--enable-javascript")
options.add_argument("--dns-prefetch-disable");
options.add_argument("--disable-gpu");
options.add_argument("--disable-blink-features=AutomationControlled")
# start the instance
driver = webdriver.Chrome(options=options)

In [3]:
def Parser(url):
    """ Opens the URL passed to the function"""
    driver.get(url)
    return(driver)
    
def Get_Properties(pages):
    """Loops through all pages of listing, and saves their info into list."""
    all_links=[]
    for i in tqdm(range(1,pages)):
        Parser(f"https://www.kv.ee/?act=search.simple&page_size=100&deal_type=1&county=1&search_type=new&parish=1061&page=[{i}]")
        try:
            #accepts the cookies
            driver.find_element_by_id("onetrust-accept-btn-handler").click()
        except:
            pass
        finally:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            properties = soup.find_all("tr", class_="object-type-apartment object-item")
            for property in properties:
                links = property.find_all("a",{"class":"object-title-a text-truncate"})
                for link in links:
                    all_links.append(link['href'])
    return(all_links)

In [4]:
##Generate the property list
links = Get_Properties(25)

  0%|          | 0/24 [00:00<?, ?it/s]

In [5]:

all_info=[]
def Properties_info(i):
            d = {}
            Parser(links[i])
            try:
                driver.find_element_by_id("onetrust-accept-btn-handler").click()
            except:
                pass
            finally:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                page_content = soup.find("div", {"class":"object-article-details"})
                price = page_content.find("div",{"class":"object-price"}).findChildren()[0]
                descr = soup.find("div",{"class":"object-article-body"}).find("p")
                aadress= soup.find('h1',{'class':'title'}).get_text(strip=True)
                #loop through rows in info table
                for items in page_content.find('table',{"class":"table-lined object-data-meta"}).find_all('tr'):
                    data = [item.get_text(strip=True) for item in items.find_all(['th','td'])]
                    d[data[0]]=data[-1]
                    d['url']=links[i]
                    d['hind']=price.get_text(strip=True)
                    d['aadress']=aadress
                    d['description'] = descr
                    d['date'] = today
            all_info.append(d)

In [6]:
for i in tqdm(range(0,len(links))):
    Properties_info(i)

  0%|          | 0/2400 [00:00<?, ?it/s]

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.561
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D0B8F3+2406643]
	Ordinal0 [0x00C9AF31+1945393]
	Ordinal0 [0x00B8C748+837448]
	Ordinal0 [0x00B7D9A0+776608]
	Ordinal0 [0x00B7C7C5+772037]
	Ordinal0 [0x00B7CDC8+773576]
	Ordinal0 [0x00B8792F+817455]
	Ordinal0 [0x00B9256B+861547]
	Ordinal0 [0x00B94F40+872256]
	Ordinal0 [0x00B7D096+774294]
	Ordinal0 [0x00B923B5+861109]
	Ordinal0 [0x00BE49CD+1198541]
	Ordinal0 [0x00BD4096+1130646]
	Ordinal0 [0x00BAE636+976438]
	Ordinal0 [0x00BAF546+980294]
	GetHandleVerifier [0x00F79612+2498066]
	GetHandleVerifier [0x00F6C920+2445600]
	GetHandleVerifier [0x00DA4F2A+579370]
	GetHandleVerifier [0x00DA3D36+574774]
	Ordinal0 [0x00CA1C0B+1973259]
	Ordinal0 [0x00CA6688+1992328]
	Ordinal0 [0x00CA6775+1992565]
	Ordinal0 [0x00CAF8D1+2029777]
	BaseThreadInitThunk [0x7733FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77BC7A7E+286]
	RtlGetAppContainerNamedOb

In [7]:
len(all_info)

19

In [120]:
filename = "data/"+today+"-dataset.csv"
pd.DataFrame(all_info).to_csv(filename)